In [1]:
# -*- coding: utf-8
# Reinaldo Chaves (@paidatocandeira)
# Acessa a base de sócios e administradores da Receita Federal
# Cruza com os dados de pagamentos da covid-19
# Separa empresas do Brasil que têm sócios os candidatos das eleições de 2018
# Verifica se um destes é sócio das empresas com pagamentos na covid-19
#

In [2]:
import pandas as pd
from datetime import datetime
import unidecode

In [3]:
# Função para retirar acentuação 
def f(str):
    return (unidecode.unidecode(str))

In [4]:
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
pagamentos_covid19 = pd.read_csv("resultados/pagamentos_covid19/pagamentos_feitos_de_empenhos_covid19_ate_03_junho_2020.csv", **kwargs)

In [6]:
pagamentos_covid19.rename(columns = {'Id Empenho':'id_empenho'},inplace = True)
pagamentos_covid19.rename(columns = {'Código Empenho':'cod_empenho'},inplace = True)

pagamentos_covid19['Valor Pago (R$)'] = pagamentos_covid19['Valor Pago (R$)'].str.replace(',','.')
pagamentos_covid19['Valor Pago (R$)'] = pagamentos_covid19['Valor Pago (R$)'].astype('float64') 
pagamentos_covid19.rename(columns = {'Valor Pago (R$)':'valor_pago_em_real'},inplace = True)

In [7]:
# Procura o Código Favorecido na Receita Federal

In [8]:
TextFileReader = pd.read_csv('../csv_dados_qsa_cnpj_28-05-20/cnpj_dados_cadastrais_pj.csv',\
                             chunksize=1000,\
                             sep='#',\
                             converters={'cnpj': lambda x: str(x)},\
                             usecols=['cnpj',
                                      'razao_social',
                                      'nome_fantasia',
                                      'situacao_cadastral',
                                      'codigo_natureza_juridica',
                                      'data_inicio_atividade',
                                      'cnae_fiscal',
                                      'uf', 
                                      'municipio'])

In [9]:


now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    # Caso deseja-se pegar o CNPJ raiz poderia criar uma nova coluna
    #df['cnpj_raiz'] = df.cnpj.str[:8]
    
    # Faz merge dos dataframes pelo número do CNPJ
    # Se por algum motivo no dataframe original existirem CNPJ duplicados faz um drop
    empresas_encontradas = pd.merge(pagamentos_covid19.drop_duplicates('Código Favorecido_x'), \
                                 df, \
                                 left_on= 'Código Favorecido_x', \
                                 right_on='cnpj')
    
    if empresas_encontradas.empty is False:
        if conta == 0:
            df_procura1 = empresas_encontradas
        else:
            df_procura1 = df_procura1.append(empresas_encontradas)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)



A extração começou às:  2020-06-05 12:55:01.319663
A extração terminou às:  2020-06-05 13:03:20.423352


In [10]:
df_procura1.shape

(2462, 112)

In [11]:
# Onde estão a maioria das empresas

In [12]:
conta = df_procura1.groupby("municipio").id_empenho.count().reset_index()
conta.sort_values('id_empenho', ascending=False)

,municipio,id_empenho
808,RIO DE JANEIRO,112
148,BRASILIA,107
924,SAO PAULO,90
763,PORTO ALEGRE,66
571,MANAUS,46
...,...,...
417,ICARA,1
418,ICATU,1
419,ICO,1
420,IGACI,1


In [13]:
df_procura1.to_csv('resultados/pagamentos_covid19_com_empresas/pagamentos_feitos_de_empenhos_covid19_com_empresas_ate_03_jun_2020.csv', index=False)

In [14]:
# Adiciona socios

In [15]:
# Porém com isso a soma com os valores não deve ser mais feita, porque cada linha é repetida sempre que mais de um sócio é encontrado

In [16]:
TextFileReader = pd.read_csv('../csv_dados_qsa_cnpj_28-05-20/cnpj_dados_socios_pj.csv',\
                             chunksize=1000,\
                             sep='#',\
                             converters={'cnpj': lambda x: str(x)},\
                             usecols=['cnpj',
                                      'identificador_socio',
                                      'nome_socio',
                                      'cnpj_cpf_socio',
                                      'cod_qualificacao_socio',
                                      'data_entrada_sociedade',
                                      'cpf_representante_legal',
                                      'nome_representante'])

In [17]:


now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    socios_encontrados = pd.merge(df_procura1, \
                                 df, \
                                 left_on='cnpj', \
                                 right_on='cnpj')
    
    if socios_encontrados.empty is False:
        if conta == 0:
            df_procura_final = socios_encontrados
        else:
            df_procura_final = df_procura_final.append(socios_encontrados)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)



A extração começou às:  2020-06-05 13:03:56.713566
A extração terminou às:  2020-06-05 13:07:16.378750


In [18]:
df_procura_final.shape

(3248, 119)

In [19]:
df_procura_final.to_csv('resultados/pagamentos_covid19_com_empresas_e_socios/pagamentos_feitos_de_empenhos_covid19_com_empresas_e_socios_ate_03_jun_2020.csv', index=False)

In [18]:
# Carrega candidatos de 2018 e 2016

In [19]:
kwargs = {'sep': ';', 'dtype': str, 'encoding': 'latin_1'}
candidatos_2018 = pd.read_csv("../candidatos_2018/consulta_cand_2018_BRASIL.csv", **kwargs)

In [20]:
# Apaga as colunas que não serão usadas
del candidatos_2018['DT_GERACAO']
del candidatos_2018['HH_GERACAO']
#del candidatos_2018['ANO_ELEICAO']
del candidatos_2018['CD_TIPO_ELEICAO']
del candidatos_2018['NM_TIPO_ELEICAO']
del candidatos_2018['NR_TURNO']
del candidatos_2018['CD_ELEICAO']
del candidatos_2018['DS_ELEICAO']
del candidatos_2018['DT_ELEICAO']
del candidatos_2018['TP_ABRANGENCIA']
del candidatos_2018['SG_UE']
del candidatos_2018['NM_UE']
del candidatos_2018['CD_CARGO']
del candidatos_2018['CD_SITUACAO_CANDIDATURA']
del candidatos_2018['CD_DETALHE_SITUACAO_CAND']
del candidatos_2018['DS_DETALHE_SITUACAO_CAND']
del candidatos_2018['TP_AGREMIACAO']
del candidatos_2018['NR_PARTIDO']
del candidatos_2018['NM_PARTIDO']
del candidatos_2018['SQ_COLIGACAO']
del candidatos_2018['NM_COLIGACAO']
del candidatos_2018['DS_COMPOSICAO_COLIGACAO']
del candidatos_2018['CD_NACIONALIDADE']
#del candidatos_2018['DS_NACIONALIDADE']
#del candidatos_2018['SG_UF_NASCIMENTO']
del candidatos_2018['CD_MUNICIPIO_NASCIMENTO']
#del candidatos_2018['NM_MUNICIPIO_NASCIMENTO']
#del candidatos_2018['NR_IDADE_DATA_POSSE']
#del candidatos_2018['NR_TITULO_ELEITORAL_CANDIDATO']
del candidatos_2018['CD_GENERO']
#del candidatos_2018['DS_GENERO']
del candidatos_2018['CD_GRAU_INSTRUCAO']
#del candidatos_2018['DS_GRAU_INSTRUCAO']
del candidatos_2018['CD_ESTADO_CIVIL']
#del candidatos_2018['DS_ESTADO_CIVIL']
del candidatos_2018['CD_COR_RACA']
#del candidatos_2018['DS_COR_RACA']
del candidatos_2018['CD_OCUPACAO']
#del candidatos_2018['NR_DESPESA_MAX_CAMPANHA']
del candidatos_2018['CD_SIT_TOT_TURNO']
#del candidatos_2018['DS_SIT_TOT_TURNO']
del candidatos_2018['NR_PROTOCOLO_CANDIDATURA']
del candidatos_2018['NR_PROCESSO ']
del candidatos_2018['NM_SOCIAL_CANDIDATO']
#del candidatos_2018['DS_SITUACAO_CANDIDATURA'] # opcional
#del candidatos_2018['ST_REELEICAO'] # opcional
#del candidatos_2018['ST_DECLARAR_BENS'] # opcional

candidatos_2018.columns = ['ano_eleicao',
'uf', 
'cargo', 
'n_sequencial', 
'numero_urna',
'nome_completo', 
'nome_urna', 
'cpf', 
'e_mail', 
'situacao_candidatura', 
'partido_eleicao', 
'nacionalidade', 
'uf_nascimento', 
'nome_municipio_nascimento',  
'data_nascimento', 
'idade_data_posse', 
'titulo_eleitoral', 
'genero', 
'grau_instrucao', 
'estado_civil', 
'cor_raca', 
'ocupacao', 
'despesa_max_campanha_informada', 
'situcao_eleicao', 
'tentou_reeleicao', 
'declarou_bens']

candidatos_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29145 entries, 0 to 29144
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ano_eleicao                     29145 non-null  object
 1   uf                              29145 non-null  object
 2   cargo                           29145 non-null  object
 3   n_sequencial                    29145 non-null  object
 4   numero_urna                     29145 non-null  object
 5   nome_completo                   29145 non-null  object
 6   nome_urna                       29145 non-null  object
 7   cpf                             29145 non-null  object
 8   e_mail                          29145 non-null  object
 9   situacao_candidatura            29145 non-null  object
 10  partido_eleicao                 29145 non-null  object
 11  nacionalidade                   29145 non-null  object
 12  uf_nascimento                   29145 non-null

In [21]:
#candidatos_2018["nome_completo"] = candidatos_2018["nome_completo"].apply(f)

In [22]:
kwargs = {'sep': ';', 'dtype': str, 'encoding': 'latin_1'}
candidatos_2016 = pd.read_csv("../candidatos_2016/consulta_cand_2016_BRASIL.csv", **kwargs)

In [23]:
# Apaga as colunas que não serão usadas
del candidatos_2016['DT_GERACAO']
del candidatos_2016['HH_GERACAO']
#del candidatos_2016['ANO_ELEICAO']
del candidatos_2016['CD_TIPO_ELEICAO']
del candidatos_2016['NM_TIPO_ELEICAO']
del candidatos_2016['NR_TURNO']
del candidatos_2016['CD_ELEICAO']
del candidatos_2016['DS_ELEICAO']
del candidatos_2016['DT_ELEICAO']
del candidatos_2016['TP_ABRANGENCIA']
del candidatos_2016['SG_UE']
del candidatos_2016['NM_UE']
del candidatos_2016['CD_CARGO']
del candidatos_2016['CD_SITUACAO_CANDIDATURA']
del candidatos_2016['CD_DETALHE_SITUACAO_CAND']
del candidatos_2016['DS_DETALHE_SITUACAO_CAND']
del candidatos_2016['TP_AGREMIACAO']
del candidatos_2016['NR_PARTIDO']
del candidatos_2016['NM_PARTIDO']
del candidatos_2016['SQ_COLIGACAO']
del candidatos_2016['NM_COLIGACAO']
del candidatos_2016['DS_COMPOSICAO_COLIGACAO']
del candidatos_2016['CD_NACIONALIDADE']
#del candidatos_2016['DS_NACIONALIDADE']
#del candidatos_2016['SG_UF_NASCIMENTO']
del candidatos_2016['CD_MUNICIPIO_NASCIMENTO']
#del candidatos_2016['NM_MUNICIPIO_NASCIMENTO']
#del candidatos_2016['NR_IDADE_DATA_POSSE']
#del candidatos_2016['NR_TITULO_ELEITORAL_CANDIDATO']
del candidatos_2016['CD_GENERO']
#del candidatos_2016['DS_GENERO']
del candidatos_2016['CD_GRAU_INSTRUCAO']
#del candidatos_2016['DS_GRAU_INSTRUCAO']
del candidatos_2016['CD_ESTADO_CIVIL']
#del candidatos_2016['DS_ESTADO_CIVIL']
del candidatos_2016['CD_COR_RACA']
#del candidatos_2016['DS_COR_RACA']
del candidatos_2016['CD_OCUPACAO']
#del candidatos_2016['NR_DESPESA_MAX_CAMPANHA']
del candidatos_2016['CD_SIT_TOT_TURNO']
#del candidatos_2016['DS_SIT_TOT_TURNO']
del candidatos_2016['NR_PROTOCOLO_CANDIDATURA']
del candidatos_2016['NR_PROCESSO ']
del candidatos_2016['NM_SOCIAL_CANDIDATO']
#del candidatos_2016['DS_SITUACAO_CANDIDATURA'] # opcional
#del candidatos_2016['ST_REELEICAO'] # opcional
#del candidatos_2016['ST_DECLARAR_BENS'] # opcional

candidatos_2016.columns = ['ano_eleicao',
'uf', 
'cargo', 
'n_sequencial', 
'numero_urna',
'nome_completo', 
'nome_urna', 
'cpf', 
'e_mail', 
'situacao_candidatura', 
'partido_eleicao', 
'nacionalidade', 
'uf_nascimento', 
'nome_municipio_nascimento',  
'data_nascimento', 
'idade_data_posse', 
'titulo_eleitoral', 
'genero', 
'grau_instrucao', 
'estado_civil', 
'cor_raca', 
'ocupacao', 
'despesa_max_campanha_informada', 
'situcao_eleicao', 
'tentou_reeleicao', 
'declarou_bens']

candidatos_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498302 entries, 0 to 498301
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ano_eleicao                     498302 non-null  object
 1   uf                              498302 non-null  object
 2   cargo                           498302 non-null  object
 3   n_sequencial                    498302 non-null  object
 4   numero_urna                     498302 non-null  object
 5   nome_completo                   498302 non-null  object
 6   nome_urna                       498302 non-null  object
 7   cpf                             498302 non-null  object
 8   e_mail                          498302 non-null  object
 9   situacao_candidatura            498302 non-null  object
 10  partido_eleicao                 498302 non-null  object
 11  nacionalidade                   498302 non-null  object
 12  uf_nascimento                 

In [24]:
frames = [candidatos_2016, candidatos_2018]

In [25]:
candidatos_2016_2018 = pd.concat(frames)

In [26]:
candidatos_2016_2018 = candidatos_2016_2018.drop_duplicates(['cpf'], keep='last')

In [27]:
candidatos_2016_2018['cpf_mascarado']='***'+candidatos_2016_2018['cpf'].str[3:9]+'**'

In [28]:
candidatos_2016_2018[['nome_completo','cpf_mascarado']].head()

,nome_completo,cpf_mascarado
0,DINIFER DE ALMEIDA SEHN,***689969**
1,DIEGO DIAS,***173330**
2,CLAUDEMIR TACÍLIO GONÇALVES,***327329**
4,IRACY AMARAL NEGRE,***308221**
5,GELSON LUIZ PETRY,***620609**


In [43]:
# Verifica que empresas como sócios politicos brasileiros

In [29]:
TextFileReader = pd.read_csv('../csv_dados_qsa_cnpj_28-05-20/cnpj_dados_socios_pj.csv',\
                             chunksize=1000,\
                             sep='#',\
                             converters={'cnpj': lambda x: str(x)},\
                             usecols=['cnpj',
                                      'identificador_socio',
                                      'nome_socio',
                                      'cnpj_cpf_socio',
                                      'cod_qualificacao_socio',
                                      'data_entrada_sociedade',
                                      'cpf_representante_legal',
                                      'nome_representante'])

In [30]:


now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    socios_encontrados = pd.merge(candidatos_2016_2018, \
                                 df, \
                                 left_on=['nome_completo', 'cpf_mascarado'], \
                                 right_on=['nome_socio','cnpj_cpf_socio'])
    
    if socios_encontrados.empty is False:
        if conta == 0:
            df_procura_socio = socios_encontrados
        else:
            df_procura_socio = df_procura_socio.append(socios_encontrados)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)



A extração começou às:  2020-05-28 08:27:43.051477
A extração terminou às:  2020-05-28 11:15:26.088636


In [31]:
df_procura_socio.shape

(171498, 35)

In [32]:
TextFileReader = pd.read_csv('../csv_dados_qsa_cnpj_28-05-20/cnpj_dados_cadastrais_pj.csv',\
                             chunksize=1000,\
                             sep='#',\
                             converters={'cnpj': lambda x: str(x)},\
                             usecols=['cnpj',
                                      'razao_social',
                                      'nome_fantasia',
                                      'situacao_cadastral',
                                      'codigo_natureza_juridica',
                                      'data_inicio_atividade',
                                      'cnae_fiscal',
                                      'uf', 
                                      'municipio'])

In [33]:


now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    # Caso deseja-se pegar o CNPJ raiz poderia criar uma nova coluna
    #df['cnpj_raiz'] = df.cnpj.str[:8]
    
    # Faz merge dos dataframes pelo número do CNPJ
    # Se por algum motivo no dataframe original existirem CNPJ duplicados faz um drop
    empresas_encontradas_polit = pd.merge(df_procura_socio, \
                                 df, \
                                 left_on= 'cnpj', \
                                 right_on='cnpj')
    
    if empresas_encontradas_polit.empty is False:
        if conta == 0:
            df_procura_polit = empresas_encontradas_polit
        else:
            df_procura_polit = df_procura_polit.append(empresas_encontradas_polit)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)



A extração começou às:  2020-05-28 11:19:19.360180
A extração terminou às:  2020-05-28 13:55:53.164307


In [34]:
df_procura_polit.shape

(171498, 43)

In [35]:
df_procura_polit.to_csv('resultados/empresas_politicos_candidatos_2016_2018_versao_28-05-20.csv', index=False)

In [ ]:
# Carrega arquivo de empresas de políticos já salvo

In [20]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
df_procura_polit = pd.read_csv("resultados/empresas_politicos_candidatos/empresas_politicos_candidatos_2016_2018_versao_28-05-20.csv", **kwargs)

In [21]:
# Verifica se há empresas de políticos entre os fornecedores da covid19

In [22]:
empresas_encontradas_polit_covid19 = pd.merge(pagamentos_covid19.drop_duplicates('Código Favorecido_x'), \
                                 df_procura_polit, \
                                 left_on= 'Código Favorecido_x', \
                                 right_on='cnpj')

In [23]:
empresas_encontradas_polit_covid19.shape

(13, 146)

In [24]:
empresas_encontradas_polit_covid19[["uf_x", "nome_urna", "cargo", "situcao_eleicao", "Data Emissão_x","cod_empenho", "id_empenho", "Órgão_x", "Unidade Gestora_x", "Código Favorecido_x", "Favorecido_x", "Valor do Empenho Convertido pra R$", "valor_pago_em_real", "Valor Restos a Pagar Inscritos (R$)", "Observação_x", "Observação_y",  "Modalidade de Licitação"]]

,uf_x,nome_urna,cargo,situcao_eleicao,Data Emissão_x,cod_empenho,id_empenho,Órgão_x,Unidade Gestora_x,Código Favorecido_x,Favorecido_x,Valor do Empenho Convertido pra R$,valor_pago_em_real,Valor Restos a Pagar Inscritos (R$),Observação_x,Observação_y,Modalidade de Licitação
0,AM,RADYR OLIVEIRA,DEPUTADO ESTADUAL,SUPLENTE,08/04/2020,160137000012020NE800064,177600344,Comando do Exército,3º BATALHAO DE INFANTARIA DE SELVA,02341467000120,AMAZONAS ENERGIA S.A,"2500,00",106.34,"0,00",ATENDER ACOES DE COMBATE AOS EFEITOS DO COVID ...,"RETENCAO-LIQUIDACAO DAS FATURAS NFS ,29582878,...",Dispensa de Licitação
1,PR,ROSANA MONNEY RIBAS,DEPUTADO ESTADUAL,SUPLENTE,08/04/2020,155913264432020NE801608,183900342,Empresa Brasileira de Serviços Hospitalares,HOSP UNIV PROF POLYDORO ERNANI DE SÃO THIAGO,26164075000100,PROVIDER PRODUTOS HOSPITALARES E SANEANTES LTDA,"3060,00","3,060.00","0,00","REEMPENHO DA SOLICITACAO 202002115, UTILIZANDO...","PAGAMENTO DA NF 1516, CONFORME SOLICITACAO DE ...",Pregão
2,TO,OGARI PACHECO,2º SUPLENTE,ELEITO,08/04/2020,155913264432020NE801615,174100368,Empresa Brasileira de Serviços Hospitalares,HOSP UNIV PROF POLYDORO ERNANI DE SÃO THIAGO,44734671000151,CRISTALIA PRODUTOS QUIMICOS FARMACEUTICOS LTDA,"697,50",17.55,"0,00","REEMPENHO DA SOLICITACAO 202002165, UTILIZANDO...","PAGAMENTO DAS NFS 2585102/2585508/2586176, CON...",Pregão
3,DF,FERNANDO MARQUES,SENADOR,NÃO ELEITO,08/04/2020,155913264432020NE801652,174200300,Empresa Brasileira de Serviços Hospitalares,HOSP UNIV PROF POLYDORO ERNANI DE SÃO THIAGO,60665981000975,UNIAO QUIMICA FARMACEUTICA NACIONAL S A,"45000,00",990.00,"0,00","REEMPENHO DA SOLICITACAO 202002362, UTILIZANDO...","PAGAMENTO DA NF 409155, CONFORME SOLICITACAO D...",Dispensa de Licitação
4,GO,LUIZ CARLOS,PREFEITO,NÃO ELEITO,04/05/2020,257001000012020NE818845,180201534,Ministério da Saúde - Unidades com vínculo di,DIRETORIA EXECUTIVA DO FUNDO NAC. DE SAUDE,08278113000165,FUNDO MUNICIPAL DE SAUDE DE RIO QUENTE,"430000,00","430,000.00","0,00",EMPENHO PARA ATENDER A PORTARIA 999 DE 30/04/2...,PAGAMENTO DE 65740-CORONAVIRUS (COVID-19) PARC...,Não se Aplica
5,AM,BIAH,VEREADOR,SUPLENTE,04/05/2020,160005000012020NE800119,176401476,Comando do Exército,54 BATALHAO DE INFANTARIA DE SELVA,31783115000123,JHONATHAN MACIEL DE SOUZA EIRELI,"2814,00","2,814.00","0,00",AQUISICAO DE MAT DE PROTECAO INDUVIDUAL PARA M...,OE04COVID00 PAGAMENTO REFERENTE A NFE-57242 AQ...,Dispensa de Licitação
6,RS,LEONARDO HOFF,PREFEITO,NÃO ELEITO,18/05/2020,160391000012020NE800140,175002246,Comando do Exército,CENTRO DE PREPARACAO OFICIAIS RESERVA/PA,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,"2000,00",956.53,"0,00",ATENDE DESP. DE FORN. DE ENERGIA ELETRICA ENFR...,"IR(1,2) 196,21 CSLL(1,0) 163,52 COFINS(3,0) 49...",Dispensa de Licitação
7,RS,LUCIO DO PRADO,DEPUTADO ESTADUAL,SUPLENTE,18/05/2020,160391000012020NE800140,175002246,Comando do Exército,CENTRO DE PREPARACAO OFICIAIS RESERVA/PA,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,"2000,00",956.53,"0,00",ATENDE DESP. DE FORN. DE ENERGIA ELETRICA ENFR...,"IR(1,2) 196,21 CSLL(1,0) 163,52 COFINS(3,0) 49...",Dispensa de Licitação
8,PI,EDENILTON,VEREADOR,NÃO ELEITO,24/04/2020,160100000012020NE800148,184901253,Comando do Exército,COMANDO DA 3A BDA DE INFANTARIA MOTORIZADA,08990041000184,RIBEIRO E DINIZ COMERCIO DE PECAS E SERVICOS A...,"1500,00","1,500.00","0,00",AQS MAT CONSUMO -DIEX REQ Nº35/PMT CIA C 24ABR...,"DANFE NR 21945, 13 MAI 20 - MATERIAL P/ MANUT...",Pregão
9,SE,MILTON ANDRADE,GOVERNADOR,NÃO ELEITO,24/04/2020,200130000012020NE800157,182701174,Departamento de Polícia Rodoviária Federal,SUPERINTENDENCIA REG. POL. RODV. FEDERAL-SE,15610595000174,UNIDADE CARDIOTORACICA DE SERGIPE LTDA,"63000,00",189.54,"0,00",EMPENHO PARA POSSIBILITAR CONTRATACAO DE SERVI...,"DARF DO PAGAMENTO DA NFS 2198, REFERENTE A SER...",Dispensa de Licitação


In [25]:
empresas_encontradas_polit_covid19.to_csv('resultados/pagamentos_covid19_empresas_candidatos/pagamentos_feitos_de_empenhos_covid19_empresas_politicos_ate_03_jun_2020.csv', index=False)

In [27]:
# Checa no arquivo original de pagamentos e também para pegar os totais de cada

In [28]:
# AMAZONAS ENERGIA S.A

In [29]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '160137000012020NE800064') &  
                           (pagamentos_covid19['id_empenho'] == '177600344')]

In [30]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
85,02341467000120,AMAZONAS ENERGIA S.A,106.34
86,02341467000120,AMAZONAS ENERGIA S.A,"1,715.50"


In [31]:
procura['valor_pago_em_real'].sum()

1821.84

In [32]:
# PROVIDER PRODUTOS HOSPITALARES E SANEANTES LTDA

In [33]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '155913264432020NE801608') &  
                           (pagamentos_covid19['id_empenho'] == '183900342')]

In [34]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
332,26164075000100,PROVIDER PRODUTOS HOSPITALARES E SANEANTES LTDA,"3,060.00"


In [35]:
# CRISTALIA PRODUTOS QUIMICOS FARMACEUTICOS LTDA 

In [36]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '155913264432020NE801615') &  
                           (pagamentos_covid19['id_empenho'] == '174100368')]

In [37]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
358,44734671000151,CRISTALIA PRODUTOS QUIMICOS FARMACEUTICOS LTDA,17.55
359,44734671000151,CRISTALIA PRODUTOS QUIMICOS FARMACEUTICOS LTDA,679.95


In [38]:
procura['valor_pago_em_real'].sum()

697.5

In [39]:
# UNIAO QUIMICA FARMACEUTICA NACIONAL S A 

In [40]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '155913264432020NE801652') &  
                           (pagamentos_covid19['id_empenho'] == '174200300')]

In [41]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
401,60665981000975,UNIAO QUIMICA FARMACEUTICA NACIONAL S A,990.00
402,60665981000975,UNIAO QUIMICA FARMACEUTICA NACIONAL S A,"44,010.00"


In [42]:
procura['valor_pago_em_real'].sum()

45000.0

In [43]:
# FUNDO MUNICIPAL DE SAUDE DE RIO QUENTE

In [44]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '257001000012020NE818845') &  
                           (pagamentos_covid19['id_empenho'] == '180201534')]

In [45]:
procura[["Data Emissão_x", "Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Data Emissão_x,Código Favorecido_x,Favorecido_x,valor_pago_em_real
2040,04/05/2020,08278113000165,FUNDO MUNICIPAL DE SAUDE DE RIO QUENTE,"430,000.00"


In [46]:
# COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ELETRICA

In [47]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '160388000012020NE800142') &  
                           (pagamentos_covid19['id_empenho'] == '174500711')]

In [48]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
6639,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,10.37
6640,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,166.90
6641,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,6.20
6642,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,100.10
6643,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,74.80
6644,08467115000100,COMPANHIA ESTADUAL DE DISTRIBUICAO DE ENERGIA ...,"1,204.27"


In [49]:
procura['valor_pago_em_real'].sum()

1562.6399999999999

In [50]:
# VWX CONSTRUCOES E SERVICOS EIRELI

In [51]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '194048192082020NE800066') &  
                           (pagamentos_covid19['id_empenho'] == '179700716')]

In [52]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
6767,14476150000180,VWX CONSTRUCOES E SERVICOS EIRELI,"7,565.68"
6768,14476150000180,VWX CONSTRUCOES E SERVICOS EIRELI,"8,489.58"
6769,14476150000180,VWX CONSTRUCOES E SERVICOS EIRELI,"2,888.72"


In [53]:
procura['valor_pago_em_real'].sum()

18943.98

In [54]:
# RIBEIRO E DINIZ COMERCIO DE PECAS E SERVICOS AUTOMOTIVO

In [55]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '160055000012020NE800042') &  
                           (pagamentos_covid19['id_empenho'] == '144306058')]

In [56]:
procura[["Data Emissão_x", "Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Data Emissão_x,Código Favorecido_x,Favorecido_x,valor_pago_em_real
9492,13/03/2020,08990041000184,RIBEIRO E DINIZ COMERCIO DE PECAS E SERVICOS A...,499.29
9493,13/03/2020,08990041000184,RIBEIRO E DINIZ COMERCIO DE PECAS E SERVICOS A...,"9,486.44"


In [57]:
procura['valor_pago_em_real'].sum()

9985.730000000001

In [58]:
# SIQUEIRA TUR TRANSPORTE E TURISMO LTDA
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '194045192082020NE800087') &  
                           (pagamentos_covid19['id_empenho'] == '181302380')]
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
5586,05929516000139,SIQUEIRA TUR TRANSPORTE E TURISMO LTDA,"2,400.00"


In [59]:
procura['valor_pago_em_real'].sum()

2400.0

In [26]:
# JHONATHAN MACIEL DE SOUZA EIRELI

In [27]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '160005000012020NE800119') &  
                           (pagamentos_covid19['id_empenho'] == '176401476')]

In [28]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
2455,31783115000123,JHONATHAN MACIEL DE SOUZA EIRELI,"2,814.00"


In [29]:
procura['valor_pago_em_real'].sum()

2814.0

In [44]:
# E também depois no site da Receita: https://servicos.receita.fazenda.gov.br/Servicos/cnpjreva/Cnpjreva_Solicitacao.asp

In [60]:
procura = pagamentos_covid19[(pagamentos_covid19['cod_empenho'] == '155913264432020NE801608') &  
                           (pagamentos_covid19['id_empenho'] == '183900342')]

In [61]:
procura[["Código Favorecido_x", "Favorecido_x", "valor_pago_em_real"]]

,Código Favorecido_x,Favorecido_x,valor_pago_em_real
243,26164075000100,PROVIDER PRODUTOS HOSPITALARES E SANEANTES LTDA,"3,060.00"


In [62]:
procura['valor_pago_em_real'].sum()

3060.0